In [ ]:
import pandas as pd
from datetime import datetime, timedelta

### Load the datasets

In [25]:
customers = pd.read_csv('customers.csv')
products = pd.read_csv('products.csv')
ratings = pd.read_csv('ratings.csv')
orders = pd.read_csv('orders.csv', parse_dates=['order_date'])


### 1. Calculate revenue for each product

In [26]:
orders['revenue'] = orders['quantity'] * products.set_index('product_id')['price']
revenue_by_product = orders.groupby('product_id')['revenue'].sum().sort_values(ascending=False)

### 2. Calculate units sold for each product

In [27]:
units_sold_by_product = orders.groupby('product_id')['quantity'].sum().sort_values(ascending=False)

### 3. Identify top performing products

In [28]:
top_performing_products_revenue = revenue_by_product.head(10)
top_performing_products_units = units_sold_by_product.head(10)

### 4. Identify top clients for the last month

In [29]:
last_month_start = datetime.now() - timedelta(days=30)
last_month_orders = orders[orders['order_date'] >= last_month_start]
top_clients_last_month = last_month_orders.groupby('customer_id')['revenue'].sum().sort_values(ascending=False).head(10)

### Print results

In [34]:
print("Top 10 Performing Products (Revenue):")
print(top_performing_products_revenue)
print("\nTop 10 Performing Products (Units Sold):")
print(top_performing_products_units)
print("\nTop 10 Clients for the Last Month:")
print(top_clients_last_month)

Top 10 Performing Products (Revenue):
product_id
27    6105.0
35    5392.0
46    4998.0
42    4200.0
1     3380.0
14    3060.0
23    3060.0
31    2876.0
29    2776.0
36    2579.0
Name: revenue, dtype: float64

Top 10 Performing Products (Units Sold):
product_id
39    80
6     77
49    63
2     63
38    62
8     62
30    61
46    60
1     59
22    59
Name: quantity, dtype: int64

Top 10 Clients for the Last Month:
Series([], Name: revenue, dtype: float64)


### Join products with revenue data

In [32]:

top_products_revenue = pd.DataFrame(top_performing_products_revenue).reset_index()
top_products_revenue = top_products_revenue.merge(products, on='product_id', how='left')

### Join products with units sold data

In [33]:

top_products_units = pd.DataFrame(top_performing_products_units).reset_index()
top_products_units = top_products_units.merge(products, on='product_id', how='left')

### Print top products with product names

In [35]:

print("\nTop 10 Products (Revenue) with Product Names:")
print(top_products_revenue[['product_name', 'revenue']])
print("\nTop 10 Products (Units Sold) with Product Names:")
print(top_products_units[['product_name', 'quantity']])


Top 10 Products (Revenue) with Product Names:
           product_name  revenue
0          Ivar Cabinet   6105.0
1  Rugveda Coffee Table   5392.0
2    Valje Wall Cabinet   4998.0
3      Småstad Wardrobe   4200.0
4           Ektorp Sofa   3380.0
5      Ingolf Bar Stool   3060.0
6        Stuva Loft Bed   3060.0
7          Nockeby Sofa   2876.0
8          Råskog Stool   2776.0
9      Gronlid Armchair   2579.0

Top 10 Products (Units Sold) with Product Names:
              product_name  quantity
0    Mackapar Shoe Storage        80
1      Brimnes Bed Storage        77
2   Söderhamn Sofa Section        63
3           Poäng Armchair        63
4  Bekant Conference Table        62
5    Melltorp Dining Table        62
6     Strandmon Wing Chair        61
7       Valje Wall Cabinet        60
8              Ektorp Sofa        59
9     Nordli Chest Drawers        59


### Join customers with top client data

In [36]:
top_clients_last_month = pd.DataFrame(top_clients_last_month).reset_index()
top_clients_last_month = top_clients_last_month.merge(customers, on='customer_id', how='left')

### Print top clients with customer names

In [37]:
print("\nTop 10 Clients for the Last Month with Customer Names:")
print(top_clients_last_month[['name', 'revenue']])


Top 10 Clients for the Last Month with Customer Names:
Empty DataFrame
Columns: [name, revenue]
Index: []


### RFM Analysis

In [38]:
import pandas as pd
from datetime import datetime

# Load the datasets
customers = pd.read_csv('customers.csv')
products = pd.read_csv('products.csv')
orders = pd.read_csv('orders.csv', parse_dates=['order_date'])

# Calculate the maximum order date
max_order_date = orders['order_date'].max()

# Calculate Recency
orders['order_date'] = pd.to_datetime(orders['order_date'])
recency = orders.groupby('customer_id').agg({'order_date': lambda x: (max_order_date - x.max()).days})
recency.columns = ['recency']

# Calculate Frequency
frequency = orders.groupby('customer_id')['order_id'].count().reset_index()
frequency.columns = ['customer_id', 'frequency']

# Calculate Monetary
monetary = orders.merge(products[['product_id', 'price']], on='product_id').groupby('customer_id')['price'].sum().reset_index()
monetary.columns = ['customer_id', 'monetary']

# Combine the three metrics into a single DataFrame
rfm = recency.merge(frequency, on='customer_id').merge(monetary, on='customer_id')

# Normalize the RFM metrics (optional)
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
rfm['recency_score'] = scaler.fit_transform(rfm[['recency']])
rfm['frequency_score'] = scaler.fit_transform(rfm[['frequency']])
rfm['monetary_score'] = scaler.fit_transform(rfm[['monetary']])

# Calculate the RFM score
rfm['rfm_score'] = (rfm['recency_score'] * 0.15) + (rfm['frequency_score'] * 0.28) + (rfm['monetary_score'] * 0.57)

# Segment customers based on RFM score
rfm['segment'] = pd.qcut(rfm['rfm_score'], q=4, labels=['Low Value', 'Moderate Value', 'High Value', 'Top Value'])

# Print the segmented customers
print(rfm)

    customer_id  recency  frequency  monetary  recency_score  frequency_score  \
0             1        0         18     12106       0.000000         0.789474   
1             2        2          9      4243       0.071429         0.315789   
2             3        4         13      7395       0.142857         0.526316   
3             4        3         22     10865       0.107143         1.000000   
4             5        8         11      7175       0.285714         0.421053   
..          ...      ...        ...       ...            ...              ...   
95           96        2          9      4458       0.071429         0.315789   
96           97        8          4      2678       0.285714         0.052632   
97           98        6          7      3699       0.214286         0.210526   
98           99        0         10      5503       0.000000         0.368421   
99          100       11         12      7327       0.392857         0.473684   

    monetary_score  rfm_sco

In [39]:
# Join ratings with products to get product names
ratings_with_product_names = ratings.merge(products[['product_id', 'product_name']], on='product_id', how='left')

# Top reviewed items by customers
top_reviewed_items = ratings_with_product_names.groupby('product_name')['rating'].count().reset_index().sort_values('rating', ascending=False)
print("Top Reviewed Items by Customers:")
print(top_reviewed_items.head(10))

# Average number of reviews per customer
avg_reviews_per_customer = ratings.groupby('customer_id')['rating'].count().reset_index().rename(columns={'rating': 'num_reviews'})
avg_reviews_per_customer = avg_reviews_per_customer.merge(customers, on='customer_id', how='left')
avg_reviews_per_customer = avg_reviews_per_customer.groupby('name')['num_reviews'].mean().reset_index().sort_values('num_reviews', ascending=False)
print("\nAverage Number of Reviews per Customer:")
print(avg_reviews_per_customer)

Top Reviewed Items by Customers:
               product_name  rating
33           Raskog Trolley      16
1   Bekant Conference Table      14
19          Lack Side Table      13
27             Nockeby Sofa      12
11         Hektar Work Lamp      11
38   Sinnerlig Pendant Lamp      11
16             Kivik Chaise      11
14             Ivar Cabinet      11
25               Micke Desk      11
5       Brimnes Bed Storage      11

Average Number of Reviews per Customer:
           name  num_reviews
80  Customer_82          9.0
39  Customer_44          9.0
73  Customer_75          9.0
91  Customer_92          8.0
4   Customer_12          8.0
..          ...          ...
36  Customer_41          1.0
11  Customer_19          1.0
64  Customer_67          1.0
76  Customer_78          1.0
69  Customer_71          1.0

[99 rows x 2 columns]
